In [1]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.
/home/jinsoo/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
L = 16000
Sec = 16000 # 1ec
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()
print(len(legal_labels))
#src folders
root_path = '/home/jinsoo'
out_path = '/home/jinsoo/jykim/cnn2'
model_path = '/home/jinsoo/jykim/cnn2'
train_data_path = os.path.join(root_path, 'KAGGLE', 'SPEECH_RECOGNITION', 'train', 'audio')
test_data_path = os.path.join(root_path, 'KAGGLE', 'SPEECH_RECOGNITION', 'test', 'audio')

12


In [3]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [4]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    pat2 = r'.+/(\w+\.' + ext + ')$'
    labels = []
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        r2 = re.match(pat2, fpath)
        if r:
            if r.group(1) in legal_labels:
                labels.append(r.group(1))
                fnames.append(r2.group(1))
            else:
                pass
        else:
            pass
    return labels, fnames

            
#     pat = r'.+/(\w+\.' + ext + ')$'
#     fnames = []
#     for fpath in fpaths:
#         r = re.match(pat, fpath)
#         if r:
#             fnames.append(r.group(1))
#     return labels, fnames


In [5]:
# def list_wavs_fname(dirpath, ext='wav'):
#     print(dirpath)
#     fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
#     pat = r'.+/(\w+)/\w+\.' + ext + '$'
#     labels = []
#     for fpath in fpaths:
#         r = re.match(pat, fpath)
#         if r:
#             labels.append(r.group(0))
#     pat = r'.+/(\w+\.' + ext + ')$'
#     fnames = []
#     for fpath in fpaths:
#         r = re.match(pat, fpath)
#         if r:
#             fnames.append(r.group(1))
#     return labels, fnames

이외의 데이터들을 unknown으로 학습시키지 말고 아예 없애버릴 예정.

In [6]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            pass
#             nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [7]:
# # labels, fnames = list_wavs_fname(train_data_path)

# new_sample_rate = 8000
# y_train = []
# x_train = []

# for label, fname in zip(labels, fnames):
#     sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
#     samples = pad_audio(samples)
#     if len(samples) > 16000:
#         n_samples = chop_audio(samples)
#     else: n_samples = [samples]
#     for samples in n_samples:
#         resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
#         _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
#         y_train.append(label)
#         x_train.append(specgram)
# x_train = np.array(x_train)
# x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
# y_train = label_transform(y_train)
# label_index = y_train.columns.values
# y_train = y_train.values
# y_train = np.array(y_train)
# del labels, fnames
# gc.collect()

In [8]:
labels, fnames = list_wavs_fname(train_data_path)
print(labels[11121], fnames[1])

/home/jinsoo/KAGGLE/SPEECH_RECOGNITION/train/audio
right 37dca74f_nohash_1.wav


In [9]:
labels, fnames = list_wavs_fname(train_data_path)

new_sample_rate = 8000
y_train = []
x_train = []

for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y_train.append(label)
        x_train.append(specgram)
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

/home/jinsoo/KAGGLE/SPEECH_RECOGNITION/train/audio


7

In [16]:
y_train.shape
# print(labels[1], fnames[1])

(20776, 11)

In [12]:
# input_shape = (99, 81, 1)
# nclass = 10
# inp = Input(shape=input_shape)
# norm_inp = BatchNormalization()(inp)
# img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
# img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.1)(img_1)
# img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
# img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# img_1 = Dropout(rate=0.1)(img_1)
# img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
# img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
# # img_1 = Dropout(rate=0.1)(img_1)
# img_1 = Flatten()(img_1)

# dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
# dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
# dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

# model = models.Model(inputs=inp, outputs=dense_1)
# opt = optimizers.Adam()

# model.compile(optimizer=opt, loss=losses.binary_crossentropy)
# model.summary()

# x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
# model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=1, shuffle=True, verbose=2)

# model.save(os.path.join(model_path, 'cnn.model'))

In [14]:
input_shape = (99, 81, 1)
nclass = 11
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy)
model.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=3, shuffle=True, verbose=2)

model.save(os.path.join(model_path, 'cnn.model'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 99, 81, 1)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 98, 80, 8)         40        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 97, 79, 8)         264       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 39, 8)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 39, 8)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 37, 16)        1168      
__________

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (20776, 1)

In [20]:
#

In [21]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('/')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fnames, imgs
    raise StopIteration()

In [22]:
# sub_names = []
# for correction in fnames:
#     correction = correction.split('/')[-1]
#     sub_names.append(correction)
    
# # fnames
# sub_names

In [23]:
# percentage_below(
#     values,
#     threshold,
#     weights=None,
#     metrics_collections=None,
#     updates_collections='unknown',
#     name=None
# )

In [25]:
# exit() #delete this
# del x_train, y_train
# gc.collect()
index = []
results = []
threshold = 0.3

threshold_count = 0
test_count = 0
def softmax_threshold (softmax_result):
#    test_count +=1
#    if (test_count >= 10):
#        sys.exit(1)
    global threshold_count
    threshold_flag = False
    for value in softmax_result:
        if (value > threshold):
            threshold_flag = True
    if (threshold_flag == True):
        p = np.argmax(softmax_result)
#        print (label_index[p])
        return label_index[p]
    if (threshold_flag == False):
        threshold_count +=1
        return 'unknown'
# print(threshold_count)

for fnames, imgs in test_data_generator(batch=32):
    predicts = model.predict(imgs)
#     predicts = np.argmax(predicts, axis=1)
#     predicts = [label_index[p] for p in predicts]
    predicts = [softmax_threshold(p) for p in predicts]
    index.extend(fnames)
    results.extend(predicts)
print(threshold_count)
df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results
df.to_csv(os.path.join(out_path, 'sub.csv'), index=False)

11179


/home/jinsoo/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:28: DeprecationWarning: generator 'test_data_generator' raised StopIteration


In [ ]:
# 17306 0.4일때
#11179 0.3일때